In [ ]:
import torch
# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

if cuda_available:
    # Get the number of GPUs available
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    # Get the name of each GPU
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")

    # Get the current GPU memory usage
    for i in range(num_gpus):
        gpu_memory_allocated = torch.cuda.memory_allocated(i)
        gpu_memory_reserved = torch.cuda.memory_reserved(i)
        print(f"GPU {i} memory allocated: {gpu_memory_allocated / (1024 ** 3):.2f} GB")
        print(f"GPU {i} memory reserved: {gpu_memory_reserved / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available.")

In [ ]:
import sys
import os
import pickle
import logging
import numba
import matplotlib
import matplotlib.pyplot as plt
# logging.getLogger('matplotlib.font_manager').disabled = True
# numba_logger = logging.getLogger('numba')
# numba_logger.setLevel(logging.WARNING)

# matplotlib_logger = logging.getLogger('matplotlib')
# matplotlib_logger.setLevel(logging.WARNING)

# Add the src directory to sys.path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(src_path)

from src.IndependentSteps import Pycromanager2H5, FFF2H5, DownloadData, Pycro

from src.SequentialSteps import BIGFISH_SpotDetection, SimpleCellposeSegmentaion, CellProperties, DilationedCytoMask ,\
                                    DisplaySequentialParams, IlluminationCorrection, Calculate_BIGFISH_Threshold, rescale_images,\
                                    Automatic_BIGFISH_Threshold

from src.FinalizationSteps import Save_Outputs, Save_Images, Save_Parameters, Save_Masks, return_to_NAS, remove_local_data_but_keep_h5, save_copy, remove_local_data

from src.Parameters import Parameters, Experiment, Settings, ScopeClass, DataContainer

from src.Displays import Display

from src.GUI import GUI, StepGUI

from src.Pipeline import Pipeline

In [ ]:
# This is how you initialize the parameters
scope = ScopeClass() 
data = DataContainer()
settings = Settings(name='IntronDiffusion_Trip_Timeseries')
experiment = Experiment()

print(experiment)
print(settings)
print(scope)
print(data)

In [ ]:
experiment.initial_data_location = [r'Users\Jack\_Datasets\IntronDiffusion\04102025_Triptolidetimeseries_JF001\JF001_0min_5uM_55']

data.local_dataset_location = None

settings.num_chunks_to_run = 1

Parameters().get_parameters()
Parameters.validate()

In [ ]:
settings.load_in_mask = True

DownloadData().run()
Pycro().run()

In [ ]:
Display().displayImage_maxProject(channel=2)

In [ ]:
experiment.FISHChannel = 0
experiment.nucChannel = 2
experiment.cytoChannel = None
experiment.voxel_size_z = 500

In [ ]:
settings.cellpose_min_size = 500
settings.cellpose_diameter = [180, 90]
settings.cellpose_pretrained_model = ["GAPDH_cyto", 'DAPI_nuclei'] 

SimpleCellposeSegmentaion().run()

In [ ]:
scope.spot_yx = 260
scope.spot_z = 1500
scope.voxel_size_yx = 100

BIGFISH_SpotDetection().run()

In [ ]:
CellProperties().run()

In [ ]:
Save_Outputs().run()
Save_Parameters().run()
Save_Images().run()
return_to_NAS().run()

settings.data_names = []
settings.copy_locations = []
save_copy().run()

remove_local_data().run()

In [ ]:
run_all = False

if run_all:
    all_datasets = [
        'Users\Jack\_Datasets\IntronDiffusion\04102025_Triptolidetimeseries_JF001\JF001_0min_5uM_55',
        'Users\Jack\_Datasets\IntronDiffusion\04102025_Triptolidetimeseries_JF001\JF001_3min_5uM_1',
        'Users\Jack\_Datasets\IntronDiffusion\04102025_Triptolidetimeseries_JF001\JF001_5min_5uM_1',
        'Users\Jack\_Datasets\IntronDiffusion\04102025_Triptolidetimeseries_JF001\JF001_10min_5uM_1',
        'Users\Jack\_Datasets\IntronDiffusion\04102025_Triptolidetimeseries_JF001\JF001_15min_5uM_2',
        'Users\Jack\_Datasets\IntronDiffusion\04102025_Triptolidetimeseries_JF001\JF001_30min_5uM_1',
        'Users\Jack\_Datasets\IntronDiffusion\04102025_Triptolidetimeseries_JF001\JF001_45min_5uM_1',
        'Users\Jack\_Datasets\IntronDiffusion\04102025_Triptolidetimeseries_JF001\JF001_60min_5uM_1',
        'Users\Jack\_Datasets\IntronDiffusion\04102025_Triptolidetimeseries_JF001\JF001_90min_5uM_1',
        'Users\Jack\_Datasets\IntronDiffusion\04102025_Triptolidetimeseries_JF001\JF001_120min_5uM_1',
        'Users\Jack\_Datasets\IntronDiffusion\04102025_Triptolidetimeseries_JF001\JF001_180min_5uM_1',
                    ]
    experiment.initial_data_location = all_datasets

    pipeline = Pipeline()
    pipeline.run_on_cluster('CLUSTER_PATH_ON_CLUSTER', name=settings.name)